In [71]:
import numpy as np
import pandas as pd
from datetime import datetime
import re

In [81]:
def seperate_date_time(x):
    date = pd.to_datetime(x.split(', ')[0], format="%m/%d/%y")
    time = x.split(', ')[1].split(' - ')[0]
    return date, time

In [85]:
def createDf(file_data, dt):
    patterns = {
        '12 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s[APap][mM]\s-\s',
        '24 Hour':'\d{1,2}/\d{1,2}/\d{2,4},\s\d{1,2}:\d{2}\s-\s',
    }
    
    data = ' '.join(file_data.read().split('\n'))
    user_message = re.split(patterns[dt], data)[1:]
    date_time = re.findall(patterns[dt], data)
    
    df = pd.DataFrame({'user_message': user_message, 'date_time': date_time})
    df[['date','time']] = df['date_time'].apply(lambda x: seperate_date_time(x)).to_list()
    users = []
    messages = []
    for message in df['user_message']:
        entry = re.split('([\w\W]+?):\s', message)
        if entry[1:]:
            users.append(entry[1])
            messages.append(entry[2])
        else:
            users.append('group_notification')
            messages.append(entry[0])

    df['user'] = users
    df['message'] = messages
    df.drop(columns=['user_message', 'date_time'], inplace=True)
    return df

In [90]:
f = open('WhatsApp Chat with Say GT (original) 12 hour format.txt', 'r', encoding='utf-8')
df = createDf(f, '12 Hour')

In [92]:
f = open('WhatsApp Chat with Say GT (original) 24 hour format.txt', 'r', encoding='utf-8')
df = pd.DataFrame(createDf(f, '24 Hour'))

In [93]:
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month_name()
df['month_num'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df['day_name'] = df['date'].dt.day_name()
df['hour'] = df['time'].apply(lambda x: x.split(':')[0])
df['minute'] = df['time'].apply(lambda x: (x.split(':')[1]).split(' ')[0])
# if '12 Hour':
#     df['meridian'] = df['time'].apply(lambda x: (x.split(':')[1]).split(' ')[1])
df.head()

,date,time,user,message,year,month,month_num,day,day_name,hour,minute
0,2022-05-09,15:01,group_notification,Messages and calls are end-to-end encrypted. N...,2022,May,5,9,Monday,15,01
1,2022-05-26,19:21,Clg Divyansh,Robotics ke notes,2022,May,5,26,Thursday,19,21
2,2022-05-26,19:44,Clg Kushagra Ahire,MST me jo numerical aaye the uske solution bhe...,2022,May,5,26,Thursday,19,44
3,2022-05-26,19:55,Clg Abbas Amjhera,<Media omitted>,2022,May,5,26,Thursday,19,55
4,2022-05-26,20:05,+91 6239 413 783,Bhai notes bhej do yrr koi 🙏🏻,2022,May,5,26,Thursday,20,05
